In [1]:
import rawpy
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from tqdm import trange
from calibrate import single_calibrate, stereo_calibrate

In [2]:
path = "test_images"
cam1 = "cam1.jpg"
cam2 = "cam2.RAF" 

In [3]:
def generate_masked_image(image, p1, p2, preview=None):
    # generate a mask
    mask = np.zeros(image.shape, dtype=np.uint8)
    mask = cv2.rectangle(mask, p1, p2, (255, 255, 255), -1)
    masked_image = cv2.bitwise_and(image, mask)
    if preview is not None:
        mask_preview = cv2.rectangle(np.zeros(image.shape, dtype=np.uint8), p1, p2, (0, 0, 255), -1)
        mask_preview = cv2.addWeighted(image, 0.8, mask_preview, 0.2, 0)
        cv2.imwrite(os.path.join(path, "ref", preview+".jpg"), mask_preview)
    return masked_image

In [4]:
cam1_p = [
    [[750, 450], [1480, 1020]],
    [[1645, 477], [2355, 991]],
    [[2491, 405], [3253, 1113]],
    [[425, 1109], [1173, 1829]],
    [[1213, 1229], [1949, 1785]],
    [[2061, 1241], [2797, 1857]],
    [[2809, 1170], [3545, 1773]],
    [[561, 1985], [1465, 2693]],
    [[1593, 2080], [2437, 2585]],
    [[2525, 1937], [3413, 2619]]
]

cam2_p = [
    [[4229, 2964], [5566, 4004]],
    [[5736, 3036], [6765, 3949]],
    [[6946, 3080], [8129, 4080]],
    [[4064, 4246], [5065, 5115]],
    [[5164, 4295], [6143, 5082]],
    [[6319, 4268], [7315, 5093]],
    [[7397, 4312], [8393, 5148]],
    [[4356, 5379], [5500, 6325]],
    [[5698, 5472], [6782, 6300]],
    [[6958, 5328], [8142, 6336]]
]

In [5]:
img1 = cv2.imread(os.path.join(path, cam1))[:3000, :4000]
img2 = rawpy.imread(os.path.join(path, cam2)).postprocess(use_camera_wb=True, no_auto_bright=True, output_bps=8)[:8733, :11644]
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
print(img1.shape, img2.shape)

image_list1 = []
image_list2 = []

for i in trange(len(cam1_p)):
    p1, p2 = cam1_p[i]
    masked_image = generate_masked_image(img1, p1, p2, f"cam1_{i}")
    image_list1.append(masked_image)
    p1, p2 = cam2_p[i]
    masked_image = generate_masked_image(img2, p1, p2, f"cam2_{i}")
    image_list2.append(masked_image)

ret, k1, d1, _, _ = single_calibrate(image_list1, path=os.path.join(path, "ref"))
print(ret)

# image_list1_resized = [cv2.resize(img, (img2.shape[1], img2.shape[0])) for img in image_list1]
k1, d1 = k1/3000*8733, d1/3000*8733

ret, k1_1, d1_1, k2, d2, R, T, E, F = stereo_calibrate(image_list1, image_list2, mtx1=k1, dist1=d1, mask = [1, 0], path=os.path.join(path, "ref"))

print(ret)


(3000, 4000, 3) (8733, 11644, 3)


100%|██████████| 10/10 [00:01<00:00,  5.15it/s]


0.385459342721133


100%|██████████| 10/10 [00:23<00:00,  2.39s/it]


3.7621684753676083


In [6]:
print(k1_1)
print(d1_1)
print(k2)
print(d2)
print(R)
print(T)


[[1.78367872e+04 0.00000000e+00 5.85852651e+03]
 [0.00000000e+00 1.78528945e+04 4.46832963e+03]
 [0.00000000e+00 0.00000000e+00 2.91100000e+00]]
[[ 5.16261726e-01 -5.81084969e-01  1.33360901e-02  4.25822892e-03
  -9.34863398e+00]]
[[8.37991326e+03 0.00000000e+00 5.88603034e+03]
 [0.00000000e+00 8.37689223e+03 4.37202108e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[ 3.75243252e-03 -2.07703589e-01  1.46416412e-05  2.79048726e-03
   1.13669298e+00]]
[[ 0.9995512   0.00426469  0.0296514 ]
 [ 0.0056359   0.94536792 -0.32595664]
 [-0.02942158  0.32597747  0.94491961]]
[[  52.08838258]
 [1350.1457852 ]
 [  91.34681247]]
